In [142]:
import json as json
import pandas as pd
import requests as requests
def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
    headers = {"Authorization": "KakaoAK 073a393656181c6073880062d3507191"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']), float(match_first['x'])

getLatLng('황태자베이커리')

IndexError: list index out of range

In [20]:
pd.read_excel('가맹점 위도경도(실패).xlsx', sheet_name='Sheet1')

,No,Unnamed: 1,주소
0,4,부산 해운대구 반여1동 1639-3 대룡빌딩 101,부산 해운대구 반여1동 1639-3 대룡빌딩 1010410홍콩반점
1,15,부산광역시 강서구 식만로,부산광역시 강서구 식만로가락한식뷔페
2,25,부산광역시 강서구 제도로,부산광역시 강서구 제도로가야정
3,29,부산광역시 사하구 감내2로 (감천동 1층),부산광역시 사하구 감내2로 (감천동 1층)감내비빔밥
4,37,부산광역시 강서구 낙동북로,부산광역시 강서구 낙동북로강동농협하나로마트
...,...,...,...
470,1578,부산광역시 동래구 석사북로,부산광역시 동래구 석사북로화성루
471,1589,부산광역시 강서구 제도로,부산광역시 강서구 제도로황금오리알
472,1593,부산광역시 남구 용주로,부산광역시 남구 용주로황성루반점
473,1595,부산광역시 사하구 하신번영로,부산광역시 사하구 하신번영로황조김밥


In [34]:
df = pd.read_excel('가맹점 위도경도(실패).xlsx', sheet_name='Sheet1')
df

,No,Unnamed: 1,주소
0,4,부산 해운대구 반여1동 1639-3 대룡빌딩 101,부산 해운대구 반여1동 1639-3 대룡빌딩 1010410홍콩반점
1,15,부산광역시 강서구 식만로,부산광역시 강서구 식만로가락한식뷔페
2,25,부산광역시 강서구 제도로,부산광역시 강서구 제도로가야정
3,29,부산광역시 사하구 감내2로 (감천동 1층),부산광역시 사하구 감내2로 (감천동 1층)감내비빔밥
4,37,부산광역시 강서구 낙동북로,부산광역시 강서구 낙동북로강동농협하나로마트
...,...,...,...
470,1578,부산광역시 동래구 석사북로,부산광역시 동래구 석사북로화성루
471,1589,부산광역시 강서구 제도로,부산광역시 강서구 제도로황금오리알
472,1593,부산광역시 남구 용주로,부산광역시 남구 용주로황성루반점
473,1595,부산광역시 사하구 하신번영로,부산광역시 사하구 하신번영로황조김밥


In [91]:
df2 = df['주소'][0:10]
df2

0    부산 해운대구 반여1동 1639-3 대룡빌딩 1010410홍콩반점
1                     부산광역시 강서구 식만로가락한식뷔페
2                        부산광역시 강서구 제도로가야정
3            부산광역시 사하구 감내2로 (감천동 1층)감내비빔밥
4                 부산광역시 강서구 낙동북로강동농협하나로마트
5       부산 부산진구 가야2동 가야공원로 52번길 21 21거창반점
6                       부산광역시 남구 석포로경명중국관
7                    부산광역시 수영구 광일로고명화돼지국밥
8                  부산광역시 북구 만덕3로 고봉김밥고봉김밥
9                     부산광역시 남구 홍곡로고봉민 김밥인
Name: 주소, dtype: object

In [97]:
for i in df2:
    print(i)

부산 해운대구 반여1동 1639-3 대룡빌딩 1010410홍콩반점
부산광역시 강서구 식만로가락한식뷔페
부산광역시 강서구 제도로가야정
부산광역시 사하구 감내2로 (감천동 1층)감내비빔밥
부산광역시 강서구 낙동북로강동농협하나로마트
부산 부산진구 가야2동 가야공원로 52번길 21 21거창반점
부산광역시 남구 석포로경명중국관
부산광역시 수영구 광일로고명화돼지국밥
부산광역시 북구 만덕3로 고봉김밥고봉김밥
부산광역시 남구 홍곡로고봉민 김밥인
